<a href="https://colab.research.google.com/github/NischalSuresh/mentalHealth_chatbot/blob/main/chatbot_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary libraries

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.

## Import necesssary packages

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset

## Load the dataset Counsel_chat from huggingface (https://github.com/nbertagnolli/counsel-chat)

In [3]:
dataset = load_dataset("nbertagnolli/counsel-chat")
dataset = dataset['train'].train_test_split(test_size=0.1)

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views'],
        num_rows: 2497
    })
    test: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views'],
        num_rows: 278
    })
})

In [5]:
len(dataset['train'])

2497

## Prompt template used in Llama-2 original model :
[INST] \<\<SYS\>\>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
\<\</SYS\>\>
{prompt}[/INST] <br>
 Source:
 - https://huggingface.co/TheBloke/Llama-2-70B-Chat-GGML
 - https://github.com/huggingface/blog/blob/main/llama2.md#fine-tuning-with-peft

 Refer chat_completion fn from generation.py for finetuning template - https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L45

## Format the text prompt to required format

In [8]:
def formatting_prompts_func(counsel_data):
  output_texts = []
  for i in range(len(counsel_data)):
    if(counsel_data['answerText'][i]) is None:
      pass
    else:
      question = (counsel_data['questionText'][i]) if (counsel_data['questionText'][i]) is not None else (counsel_data['questionTitle'][i])
      text = f"<s>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\n### Question: {question.strip()}\n### Answer: {(counsel_data['answerText'][i]).strip()}</s>"
      output_texts.append(text)
  return output_texts

## Check data for error (missing entries)

In [ ]:
print(len(dataset['train']))
for i in range(len(dataset['train'])):
  try:
    question = (dataset['train']['questionText'][i]) if (dataset['train']['questionText'][i]) is not None else (dataset['train']['questionTitle'][i])
    answer = (dataset['train']['answerText'][i])
  except:
    print("error",i)


In [8]:
out = formatting_prompts_func(dataset['train'])

## check data output

In [9]:
print(out[1])

</s>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
### Question: I have been thinking a lot about certain situations and having my worries about what others may think or say.
### Answer: I think it's a great idea that you are asking this question now while it's early on. The first thing I wonder is what age you are.  At different transitions and stages in life, it's really common for anxieties to come out of the woodwork.  First time this takes place at a greater level is during adolescence, because a teen's "job" is to figure out who they are within the context of other people.  Spikes in anxiety also can also occur after high school when you are leaving the nest and heading to college, or the late twenties when it feels like you are expected to have your life figured out and compare to where others are in their lives.  That same scenario can show up in mid-life as well for different reasons depending on the person.  Aside

## checking Llama format from llama-recipes

In [ ]:
!pip install llama-recipes

In [ ]:
from llama_recipes.utils.dataset_utils import get_preprocessed_dataset
from llama_recipes.configs.datasets import samsum_dataset

temp_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')

In [ ]:
temp_dataset

In [ ]:
tokenizer.decode(temp_dataset[1]['input_ids'])

## Finetune

In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
model_name = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    trust_remote_code = True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [15]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 200
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [16]:
from trl import SFTTrainer
max_seq_length = 512
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset['train'],
    peft_config = peft_config,
    formatting_func = formatting_prompts_func,
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = training_arguments
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/2497 [00:00<?, ? examples/s]

In [17]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [18]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.376600
20,2.009200
30,1.732400
40,1.513900
50,1.237700
60,0.929700
70,0.623500
80,0.392300
90,0.236700
100,0.111600


TrainOutput(global_step=200, training_loss=0.5738980039954186, metrics={'train_runtime': 3481.2428, 'train_samples_per_second': 0.919, 'train_steps_per_second': 0.057, 'total_flos': 3.683028775624704e+16, 'train_loss': 0.5738980039954186, 'epoch': 100.0})

In [27]:
sample_question = "My husband left me after 10 years of marriage. What should I do?"
text = f"<s>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\n### Question: {sample_question}\n### Answer: "
print(text)
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
raw_output = (tokenizer.decode(outputs[0], skip_special_tokens=True))
print(raw_output.strip().split('###')[1:3])

<s>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
### Question: My husband left me after 10 years of marriage. What should I do?
### Answer: 


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


[' Question: My husband left me after 10 years of marriage. What should I do?\n', " Answer: 3 Things to Do First:1. Take care of yourself (and be patient)2. Don't make any decisions yet (except to stick around and keep breathing!)3. Find a way to explore this space with someone you can"]


In [26]:
print(final_output)

[' Question: My husband left me after 10 years of marriage. What should I do?\n', ' Answer: 3 Things to Do When Your Husband Leaves YouAfter your husband has been a witness to you being a person who is trying to be a healthy helpful person. 3. Find another person who will help you describe the type of ways you would like to be supported. There are lots of ways to get support when there has been a long time in a way of life that has been challenging your health. If you go alone it may just be a scary silent journey. Not only']


## Save the adapter to HF Hub

In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [28]:
trainer.push_to_hub("llama2_finetuned_counselChat")

'https://huggingface.co/nischal95/results/tree/main/'